`Course Instructor`: **John Chiasson**

`Author (TA)`: **Ruthvik Vaila**
# Note:
* In this notebook we shall use an iterator to feed `.tfrecord` files to a model.
* Python `Lists`, `Tuples` are examples of iterators. These are data structures on which one can iterate. 
* Learn more about Python [generator](https://towardsdatascience.com/python-generators-393455aa48a3), [iterators](https://anandology.com/python-practice-book/iterators.html).
* Keep an eye on RAM usage when compared to generator way of training a NN.
* Tested on `Python 3.7.5` with `Tensorflow 1.15.0` and `Keras 2.2.4`. 
* Tested on `Python 2.7.17` with `Tensorflow 1.15.3` and `Keras 2.2.4`.

# Imports

In [1]:
import sys
sys.version

'3.7.5 (default, Nov  7 2019, 10:50:52) \n[GCC 8.3.0]'

In [2]:
import tensorflow as tf
#tf.compat.v1.enable_eager_execution()
from tensorflow.python import keras as keras
from tensorflow.python.client import device_lib
import numpy as np
import sys, pickle, os, IPython
import h5py, time, inspect
import IPython.display as display

config = tf.ConfigProto()
config.gpu_options.allow_growth=True

In [3]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 884131572435183030,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 7914684519199585740
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 2676915676734791001
 physical_device_desc: "device: XLA_GPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 7400253031
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 6483027937895957132
 physical_device_desc: "device: 0, name: GeForce RTX 2080 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 7.5"]

In [3]:
tf.__version__

'1.15.0'

In [4]:
keras.__version__

'2.2.4-tf'

# Parser

In [5]:
def parser(record):
    image_feature_description = {
    'label': tf.io.FixedLenFeature([], tf.int64),
    'image_raw': tf.io.VarLenFeature(tf.float32),
    }
    parsed = tf.io.parse_single_example(record, image_feature_description)
    
    image = parsed['image_raw']
    image = tf.sparse.to_dense(image,default_value = 0)
    label = tf.cast(parsed["label"], tf.int32)
    
    #return {"image_data": image}, label
    return image, label
    

# Create a `.tfrecord` `dataset` iterator and 
* [Source](https://medium.com/@moritzkrger/speeding-up-keras-with-tfrecord-datasets-5464f9836c36)

In [6]:
def create_dataset_iterator(path='',shuffle_buffer_size=None, batch_size=None, compression='GZIP', mode='train'):
    filenames = [file for file in os.listdir(os.path.join(os.getcwd(), path)) if file.endswith('.tfrecord')]
    filenames = [os.path.join(path, file) for file in filenames]
    
    # This works with arrays as well
    dataset = tf.data.TFRecordDataset(filenames, compression_type=compression)
    
    # Maps the parser on every filepath in the array. You can set the number of parallel loaders here
    dataset = dataset.map(parser, num_parallel_calls=8)
    
    # This dataset will go on forever
    dataset = dataset.repeat()
        
    # Set the number of datapoints you want to load and shuffle
    #if(mode=='train'):
    
    dataset = dataset.shuffle(shuffle_buffer_size)
    
    # Set the batchsize
    dataset = dataset.batch(batch_size)
    
    # Create an iterator
    iterator = dataset.make_one_shot_iterator()
    
    # Create your tf representation of the iterator
    image, label = iterator.get_next()

    # Bring your picture back in shape
    image = tf.reshape(image, [-1, 3630])
    
    # Create a one hot array for your labels
    label = tf.one_hot(label, 47)
    
    return image, label

# A small model

In [7]:
#Combine it with keras
def smol_model(inputs=None, outputs=None):
    model_input = keras.layers.Input(tensor=inputs)

    #Build your network
    model_output = keras.layers.Dense(1500,activation='relu')(model_input)
    model_output = keras.layers.Dense(47,activation='softmax')(model_output)

    #Create your model
    train_model = keras.models.Model(inputs=model_input, outputs=model_output)

    #Compile your model
    train_model.compile(optimizer=keras.optimizers.RMSprop(lr=0.0001),
                        loss='mean_squared_error',
                        metrics=['accuracy'],
                        target_tensors=[outputs])
    return train_model

# `Batch_Size` and `Buffer_Size` setting

In [8]:
BATCH_SIZE = 32
SHUFFLE_BUFFER = BATCH_SIZE**2
EPOCHS = 3
STEPS_PER_EPOCH = 112799 / BATCH_SIZE # number of batches per epoch.

# Start training the model

In [9]:
#Get your datatensors
image, label = create_dataset_iterator('40_tfrecords/train',SHUFFLE_BUFFER, BATCH_SIZE)
train_model = smol_model(inputs=image, outputs=label)
train_model.summary()

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3630)]            0         
_________________________________________________________________
dense (Dense)                (None, 1500)              5446500   
_________________________________________________________________
dense_1 (Dense)              (None, 47)                70547     
Total params: 5,517,047
Trainable params: 5,517,047
Non-trainable params: 0
_________________________________________________________________


* Samples is number of batches per epoch.
* If you don't specify `steps_per_epoch=STEPS_PER_EPOCH` then you get below error
* `ValueError: When using data tensors as input to a model, you should specify the` `steps_per_epoch` `argument.`

In [10]:
train_model.fit(epochs=EPOCHS, max_queue_size=1000, steps_per_epoch=STEPS_PER_EPOCH)

Train on 3524.96875 samples
Epoch 1/3
3525/3524 [==============================] - 15s 4ms/step - loss: 0.0096 - acc: 0.6967
Epoch 2/3
3525/3524 [==============================] - 15s 4ms/step - loss: 0.0049 - acc: 0.8519
Epoch 3/3
3525/3524 [==============================] - 15s 4ms/step - loss: 0.0034 - acc: 0.8986


# Test the model

In [11]:
BATCH_SIZE = 32
STEPS = 18800/BATCH_SIZE  # number of batches per epoch.
SHUFFLE_BUFFER = BATCH_SIZE**2
image, label = create_dataset_iterator('40_tfrecords/test',SHUFFLE_BUFFER, BATCH_SIZE)

* Evaluating at small batch sizes (same as training)

In [12]:
train_model.evaluate(image, label, steps=STEPS)

588/587 [==============================] - 1s 2ms/step - loss: 0.0029 - acc: 0.9132


[0.0029322847968618823, 0.913159]

* Evaluating all of the test data at once.

In [13]:
BATCH_SIZE = 18800
STEPS = 18800/BATCH_SIZE  # number of batches per epoch.
SHUFFLE_BUFFER = BATCH_SIZE**2
image, label = create_dataset_iterator('40_tfrecords/test',SHUFFLE_BUFFER, BATCH_SIZE)

In [14]:
train_model.evaluate(image, label, steps=STEPS)

1/1 [==============================] - 0s 3ms/step - loss: 0.0019 - acc: 0.9375


[0.001930729136802256, 0.9375]


* There is a small discrepency in the two accuracies, try to find out why is that happening. 

# Restart the notebook to free up the `GPU` and `RAM`

In [4]:
IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel

{'status': 'ok', 'restart': True}

# Exercise: Log the RAM usage for the three cases.
* How the RAM usage varies when training the model using an iterators of `.tfrecord` vs direct `NumPy` array vs using a generator of `tfrecords`.
* How does the speed vary?
* Plot various metrics like `training cost vs epochs`, `training accuracy vs epochs` etc. These metrics can be found in the dictionary `history` returned by the model.